## Chat bot project

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [6]:
import os

new_directory = "/content/drive/MyDrive/medical_bot"
os.chdir(new_directory)

In [17]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from tqdm.autonotebook import tqdm


In [18]:
PINECONE_API_KEY = Pinecone_api_key

In [19]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [20]:
!pwd

/content/drive/MyDrive/medical_bot


In [24]:
extracted_data = load_pdf("/content/drive/MyDrive/medical_bot/")

In [25]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [26]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [27]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [30]:
embeddings = download_hugging_face_embeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

# Making connection

In [32]:
pc = Pinecone(
      api_key=PINECONE_API_KEY
    )

# Now do stuff
if 'medical-chatbot' not in pc.list_indexes().names():
  pc.create_index(
    name='medical-chatbot',
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
      cloud='gcp-starter',
      region='us-central1'
        )
  )

In [33]:
index_name = "medical-chatbot"
index=pc.Index(index_name)

In [34]:
def vector(texts):
    vectors = []
    for i in range(len(texts)):
        vector = embeddings.embed_query(texts[i].page_content)
        if isinstance(vector,list):
          vector_obj={
              'id':str(i),
              'values':vector,
              'metadata':{'text':texts[i].page_content}
          }
          vectors.append(vector_obj)
        else:
          print("Invalid")
    return vectors

In [35]:

vectors = vector(text_chunks)

In [36]:
len(vectors)

7020

## Loading data on pinecone

In [37]:
batch_size=50
for i in range(0, len(vectors), batch_size):
  batch = vectors[i:i + batch_size]
  # Print the type of the batch and the type of a sample vector
  #print(f'Type of batch: {type(batch)}')
  sample_vector = batch[0]
  #print(f"Type of vector for id {sample_vector['id']} : {type(sample_vector['values'])}")
  #print(batch) # print the batch to verify the format
  index.upsert(vectors=batch)

In [38]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0702,
 'namespaces': {'': {'vector_count': 7020}},
 'total_vector_count': 7020}

## Initializing vector store

In [42]:
pip install -qU langchain-pinecone==0.0.3

In [43]:
from langchain_pinecone import PineconeVectorStore
text_field = "text"
vectorstore = PineconeVectorStore(
    index, embeddings, text_field
)

In [44]:
query = "What are Allergies"

In [45]:
vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"),
 Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly cause al

In [46]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [47]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [48]:
model='/content/drive/MyDrive/medical_bot/llama-2-7b-chat.ggmlv3.q4_0.bin'

In [51]:
llm=CTransformers(model=model,
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [52]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [54]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt:"What are Allergies"
Response :  Allergies are abnormal reactions of the immune system that occur in response to otherwise harmless substances.


KeyboardInterrupt: Interrupted by user